<a href="https://colab.research.google.com/github/thofes/judging_SJ_BA/blob/main/Mark10_movement_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Connect GDrive
from google.colab import drive
drive.mount('/content.gdrive')


Mounted at /content.gdrive


In [29]:
import json
import numpy as np
import cv2

with open('/content.gdrive/MyDrive/Test2/mark5/predictions.json', "r") as fp:
    data = json.load(fp)

print(type(data["x_center"]))
print(data["x_center"])
# convert lists to numpy arrays
x_center = np.array(data["x_center"])
y_center = np.array(data["y_center"])
x1 = np.array(data["x1"])
x2 = np.array(data["x2"])
y1 = np.array(data["y1"])
y2 = np.array(data["y2"])
det = np.array(data["det"])
width = 1920
hight = 1080


<class 'list'>
[1276.4999771118164, 1277.9999542236328, 347.49999046325684, 1145.0000381469727, 416.49999618530273, 956.0000038146973, 720.9999847412109, 1232.4999618530273, 346.5000057220459, 416.49999618530273, 1276.9999694824219, 1144.5000457763672, 956.9999885559082, 720.4999923706055, 1456.4999771118164, 416.49999618530273, 1276.4999771118164, 1145.5000305175781, 956.0000038146973, 347.49999046325684, 720.4999923706055, 1155.4999923706055, 1275.999984741211, 956.4999961853027, 416.00000381469727, 1145.5000305175781, 348.0000114440918, 720.0, 350.50000190734863, 416.49999618530273, 1275.4999923706055, 956.0000038146973, 1145.0000381469727, 1115.0000381469727, 720.4999923706055, 1095.999984741211, 1276.9999694824219, 350.50000190734863, 416.9999885559082, 956.4999961853027, 1145.0000381469727, 719.5000076293945, 1064.000015258789, 956.9999885559082, 1276.4999771118164, 350.50000190734863, 1145.5000305175781, 416.49999618530273, 720.0, 837.4999809265137, 1277.4999618530273, 956.99998

In [42]:
print(sum(det[:1]))
print(det)
print(sum(det[:0+1]))
print(sum(det[:0])+0)
print(math.sqrt((x_center[sum(det[:0+1])]-x_center[sum(det[:0])+0])**2 + (y_center[sum(det[:0+1])]-y_center[sum(det[:0])+0])**2))

list_soll = [1,0,'X',0,5,0,0,7,6,2,4,6,2,2,5,0,1,4,1,0,2]


7
[7 7 7 7 7 7 7 8 7 7 8 8 8 6 9 6 8 8 8 7 9]
7
0
115.69361771412606


In [36]:
#detect movement
import math
from collections import OrderedDict

athlet_det_index = []
y_centers = []

for y in range(21):
    distance = []
    distance_index = []
    valid_dist_index = []
    y_center_per_det = []
    if y < 20:
        for x in range(det[y+1]):
            y_center_per_det.append(y_center[sum(det[:y])+x])
            for i in range(det[y]):
                distance.append(math.sqrt((x_center[sum(det[:y+1])+x]-x_center[sum(det[:y])+i])**2 + (y_center[sum(det[:y+1])+x]-y_center[sum(det[:y])+i])**2))
                distance_index.append(i)
        #print(distance)
        
              
    else:
        for x in range(det[y-1]):

            for i in range(det[y]):
                distance.append(math.sqrt((x_center[sum(det[:y-1])+x]-x_center[sum(det[:y])+i])**2 + (y_center[sum(det[:y-1])+x]-y_center[sum(det[:y])+i])**2))
                distance_index.append(i)
        #print(distance)
        #print(distance_index)

    #print(y_center_per_det)
    y_centers.append(y_center_per_det)
    for i, x in enumerate(distance):
        if x > 15 and x < 50:
            valid_dist_index.append(distance_index[i])
    
    #print(y_centers)
    #print(len(y_center_per_det))
    #print(athlet_det_index)
    print("bevor", valid_dist_index)
    #print("athlet", athlet_det_index)
    if len(valid_dist_index) == 0:
        athlet_det_index.append('X')
        """
        #athlet_det_index.append('X')
        distance = []
        distance_index = []
        valid_dist_index = []
        y_center_per_det = []        
        for x in range(det[y-1]):

            for i in range(det[y]):
                distance.append(math.sqrt((x_center[sum(det[:y-1])+x]-x_center[sum(det[:y])+i])**2 + (y_center[sum(det[:y-1])+x]-y_center[sum(det[:y])+i])**2))
                distance_index.append(i)
        #y_centers.append(y_center_per_det)
        for i, x in enumerate(distance):
            if x > 15 and x < 50:
                valid_dist_index.append(distance_index[i])
        print("BPBP", valid_dist_index)
        """
        

    if len(valid_dist_index) > 1:
        valid_dist_index = list(OrderedDict.fromkeys(valid_dist_index))
        #print(valid_dist_index) 
    #print(len(valid_dist_index))
    if len(valid_dist_index) == 1:
        athlet_det_index.append(valid_dist_index[0])
      



    if len(valid_dist_index) > 1:
        dist_to_previous_y_center = []
        print("for", valid_dist_index)
        for i, x in enumerate(valid_dist_index):
            #print("x",x)
            #print("y",y)
            #print(len(y_centers))
            #print(len(y_center_per_det))
            #print(len(athlet_det_index))
            #print("abs", abs(y_center_per_det[x]-y_centers[y-1][athlet_det_index[y-1]]))
            dist_to_previous_y_center.append(abs(y_center_per_det[x]-y_centers[y-1][athlet_det_index[y-1]]))

        #print("dist", dist_to_previous_y_center)

        closest = min(dist_to_previous_y_center)
        min_index = dist_to_previous_y_center.index(closest)
        athlet_det_index.append(valid_dist_index[min_index])
        #print("BP",valid_dist_index[min_index])




    






#print(list_soll)
print(athlet_det_index)







bevor [1]
bevor []
bevor []
bevor [0]
bevor [5]
bevor [0]
bevor [0]
bevor [7]
bevor [6]
bevor [5, 2]
for [5, 2]
bevor [6, 0, 4]
for [6, 0, 4]
bevor [1, 6, 0]
for [1, 6, 0]
bevor [2]
bevor [2, 2]
bevor [0, 5]
for [0, 5]
bevor [0]
bevor [4, 0, 1]
for [4, 0, 1]
bevor [3, 4, 0]
for [3, 4, 0]
bevor [1, 0]
for [1, 0]
bevor [3, 0]
for [3, 0]
bevor [2, 1]
for [2, 1]


IndexError: ignored

In [32]:
for i,x in enumerate(athlet_det_index):
    if x != 'X':
        print(sum(det[:i]))
        print(x)
        x_1 = x1[sum(det[:i])+x]-50
        x_2 = x2[sum(det[:i])+x]+50
        y_1 = y1[sum(det[:i])+x]-50
        y_2 = y2[sum(det[:i])+x]+50

        if x_1 < 0:
            x_1 = 0

        if x_2 > width:
            x_2 = width

        if y_1 < 0:
            y_1 = 0

        if y_2 > hight:
            y_2 = hight


        path = '/content.gdrive/MyDrive/Test2/' + str(i+1) + '.jpg'
        print(path)
        im = cv2.imread(path)
        print(x_1, x_2, y_1, y_2)
        crop = im[int(y_1):int(y_2), int(x_1):int(x_2)] 
        #save_dir.parent.mkdir(parents=True, exist_ok=True)  # make directory
        save_path = '/content.gdrive/MyDrive/Test2/mark10/' + str(i+1) + '.jpg'
        cv2.imwrite(save_path, crop) # speicher nur den zugeschnittenen Springer


0
1
/content.gdrive/MyDrive/Test2/1.jpg
1204.0 1352.0 543.0 749.0
21
0
/content.gdrive/MyDrive/Test2/4.jpg
1074.0 1237.0 562.0 765.0
28
5
/content.gdrive/MyDrive/Test2/5.jpg
1032.0 1198.0 551.0 767.0
35
0
/content.gdrive/MyDrive/Test2/6.jpg
1007.0 1185.0 578.0 765.0
42
0
/content.gdrive/MyDrive/Test2/7.jpg
974.0 1154.0 581.0 773.0
49
7
/content.gdrive/MyDrive/Test2/8.jpg
936.0 1093.0 584.0 779.0
57
6
/content.gdrive/MyDrive/Test2/9.jpg
906.0 1056.0 584.0 784.0
64
2
/content.gdrive/MyDrive/Test2/10.jpg
874.0 1022.0 596.0 786.0
71
4
/content.gdrive/MyDrive/Test2/11.jpg
837.0 986.0 601.0 788.0
79
6
/content.gdrive/MyDrive/Test2/12.jpg
804.0 952.0 595.0 790.0
87
2
/content.gdrive/MyDrive/Test2/13.jpg
776.0 918.0 619.0 794.0
95
2
/content.gdrive/MyDrive/Test2/14.jpg
737.0 881.0 610.0 794.0
101
5
/content.gdrive/MyDrive/Test2/15.jpg
701.0 853.0 618.0 792.0
110
0
/content.gdrive/MyDrive/Test2/16.jpg
671.0 820.0 620.0 792.0
116
1
/content.gdrive/MyDrive/Test2/17.jpg
631.0 785.0 619.0 794.0
124